In [1]:
import pandas as pd
import numpy as np
import surprise

In [2]:
ratings = pd.read_csv("ratings.txt",sep=' ',names = ['uid','iid','rating'])
ratings.head()

,uid,iid,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


In [3]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))

Ratings range between 0.5 and 4.0


Converting the data into **surprise** format:

In [4]:
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

**Similarity options:**

In case of Item based filtering, `user_based` value will be `False`

In [5]:
similarity_options = {'name': 'cosine', 'user_based': False}

Default K = 40

In [6]:
algo = surprise.KNNBasic(sim_options = similarity_options)
output = algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


The above `.fit()` calculates expected rating for all the users

Say we want expected rating of user=50 for item 217

In [7]:
pred = algo.predict(uid='50',iid='217')
pred

Prediction(uid='50', iid='217', r_ui=None, est=3.0028030537791928, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [8]:
score = pred.est
print(score)


3.0028030537791928


All the users

In [9]:
iids = ratings['iid'].unique()
iids

array([   1,    2,    3, ..., 2069, 2070, 2071], dtype=int64)

In [10]:
len(iids)

2071

Consider ratings given by user=50

In [11]:
user_no=50

In [12]:
rec_u = ratings[ratings['uid'] == user_no ]
iids_u = rec_u['iid']
print("List of iid that uid={0} has rated:".format(user_no))
print(iids_u.values)

List of iid that uid=50 has rated:
[  8 211   3   2 219 234  12 254 250 207  11 253 236  84  10   7 233  13
   1   5   6 252 241 216 257 206   4 217   9 215 213  17 255 220 121 245
 239 251 235]


In [13]:
iids_to_predict = np.setdiff1d(iids,iids_u)
print("List of iid which uid={0} did not rate(in all {1}) :".format(user_no,len(iids_to_predict)))
print(iids_to_predict)

List of iid which uid=50 did not rate(in all 2032) :
[  14   15   16 ... 2069 2070 2071]


In [14]:
len(iids_to_predict)

2032

Create a testset for getting the expected rating

In [15]:
testset = [[user_no,iid,0.] for iid in iids_to_predict]
testset

[[50, 14, 0.0],
 [50, 15, 0.0],
 [50, 16, 0.0],
 [50, 18, 0.0],
 [50, 19, 0.0],
 [50, 20, 0.0],
 [50, 21, 0.0],
 [50, 22, 0.0],
 [50, 23, 0.0],
 [50, 24, 0.0],
 [50, 25, 0.0],
 [50, 26, 0.0],
 [50, 27, 0.0],
 [50, 28, 0.0],
 [50, 29, 0.0],
 [50, 30, 0.0],
 [50, 31, 0.0],
 [50, 32, 0.0],
 [50, 33, 0.0],
 [50, 34, 0.0],
 [50, 35, 0.0],
 [50, 36, 0.0],
 [50, 37, 0.0],
 [50, 38, 0.0],
 [50, 39, 0.0],
 [50, 40, 0.0],
 [50, 41, 0.0],
 [50, 42, 0.0],
 [50, 43, 0.0],
 [50, 44, 0.0],
 [50, 45, 0.0],
 [50, 46, 0.0],
 [50, 47, 0.0],
 [50, 48, 0.0],
 [50, 49, 0.0],
 [50, 50, 0.0],
 [50, 51, 0.0],
 [50, 52, 0.0],
 [50, 53, 0.0],
 [50, 54, 0.0],
 [50, 55, 0.0],
 [50, 56, 0.0],
 [50, 57, 0.0],
 [50, 58, 0.0],
 [50, 59, 0.0],
 [50, 60, 0.0],
 [50, 61, 0.0],
 [50, 62, 0.0],
 [50, 63, 0.0],
 [50, 64, 0.0],
 [50, 65, 0.0],
 [50, 66, 0.0],
 [50, 67, 0.0],
 [50, 68, 0.0],
 [50, 69, 0.0],
 [50, 70, 0.0],
 [50, 71, 0.0],
 [50, 72, 0.0],
 [50, 73, 0.0],
 [50, 74, 0.0],
 [50, 75, 0.0],
 [50, 76, 0.0],
 [50, 77

Generate predictions on testset

In [16]:
predictions = algo.test(testset)
predictions[1000]

Prediction(uid=50, iid=1040, r_ui=0.0, est=3.540093440208082, details={'actual_k': 38, 'was_impossible': False})

In [17]:
type(predictions)

list

In [18]:
pred_ratings = np.array([pred.est for pred in predictions])
pred_ratings

array([3.60714286, 3.52692492, 3.56540385, ..., 3.75      , 3.75      ,
       3.46875   ])

In [19]:
iids_to_predict

array([  14,   15,   16, ..., 2069, 2070, 2071], dtype=int64)

In [20]:
df_exp_ratings = pd.DataFrame({'iid':iids_to_predict, 'exp_rating':pred_ratings})
df_exp_ratings.head()

,iid,exp_rating
0,14,3.607143
1,15,3.526925
2,16,3.565404
3,18,3.607143
4,19,3.607143


In [21]:
df_exp_ratings.sort_values('exp_rating', ascending=False)

,iid,exp_rating
218,258,4.000000
1805,1845,4.000000
1766,1806,4.000000
1167,1207,3.833333
1165,1205,3.785714
...,...,...
1826,1866,3.000000
1737,1777,3.000000
1736,1776,3.000000
1169,1209,2.500000


Top 10 Items' Expected ratings

In [22]:
df_exp_ratings.sort_values('exp_rating', ascending=False).head(10)

,iid,exp_rating
218,258,4.000000
1805,1845,4.000000
1766,1806,4.000000
1167,1207,3.833333
1165,1205,3.785714
2028,2068,3.750000
896,936,3.750000
1689,1729,3.750000
2030,2070,3.750000
2029,2069,3.750000


In [40]:
######## Tuning with different K, sim options - user based and item based with different similarity metrics

In [23]:
np.arange(30,110,20)

array([30, 50, 70, 90])

In [43]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

param_grid = {'k': np.arange(10,60,10),'sim_options':{'name':['cosine','pearson_baseline','MSD'],'user_based':[False,True]}}
kfold = KFold(n_splits=5, random_state=24, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid, measures=['rmse', 'mae'], cv=kfold)

Running the Grid Search CV

In [44]:
gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estima

**Best Score**

In [45]:
print(gs.best_score['rmse'])

0.8177583813567078


**Best Param**

In [46]:
print(gs.best_params['rmse'])

{'k': 20, 'sim_options': {'name': 'MSD', 'user_based': False}}


In [ ]:
#### KNNWithZScore

In [47]:
param_grid = {'k': np.arange(10,60,10),'sim_options':{'name':['cosine','pearson_baseline','MSD'],'user_based':[False,True]}}
kfold = KFold(n_splits=5, random_state=24, shuffle=True)
gs = GridSearchCV(surprise.KNNWithZScore, param_grid, measures=['rmse', 'mae'], cv=kfold)

gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estima

In [48]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8180801297834872
{'k': 50, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}}


In [49]:
#### KNN  SVD  #############

In [51]:
param_grid = {'n_epochs': np.arange(50,150,10),'lr_all':np.linspace(0.001,1,5),'reg_all':np.linspace(0.01,10.8,5),'n_factors':[50,100]}
kfold = KFold(n_splits=5, random_state=24, shuffle=True)
gs = GridSearchCV(surprise.SVD, param_grid, measures=['rmse', 'mae'], cv=kfold, n_jobs=-1)

gs.fit(data)

In [52]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8034208037655631
{'n_epochs': 100, 'lr_all': 0.001, 'reg_all': 0.01, 'n_factors': 50}


In [ ]:
######## SVD++ (l1 regularizor)

In [54]:
param_grid = {'n_epochs': np.arange(50,150,10),'lr_all':np.linspace(0.001,1,5),'reg_all':np.linspace(0.01,10.8,5),'n_factors':[50,100]}
kfold = KFold(n_splits=5, random_state=24, shuffle=True)
gs = GridSearchCV(surprise.SVDpp, param_grid, measures=['rmse', 'mae'], cv=kfold, n_jobs=-1)

gs.fit(data)

KeyboardInterrupt: 

In [ ]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])